In [2]:
import pygame
import random
import math
import sys
import numpy as np

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 1000, 700
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Advanced Space Shooter")

# Colors
BACKGROUND = (5, 5, 25)
PLAYER_COLOR = (0, 200, 255)
ENEMY_COLORS = [(255, 50, 50), (50, 255, 50), (255, 200, 50), (200, 50, 255)]
BULLET_COLOR = (255, 255, 200)
POWERUP_COLORS = [(255, 215, 0), (0, 255, 255), (255, 105, 180)]
PARTICLE_COLORS = [(255, 255, 200), (255, 100, 100), (100, 200, 255)]
UI_COLOR = (100, 200, 255)
UI_BG = (10, 20, 40, 180)

# Game variables
clock = pygame.time.Clock()
FPS = 60
score = 0
level = 1
game_over = False
paused = False
stars = []

# Create background stars
for _ in range(150):
    x = random.randint(0, WIDTH)
    y = random.randint(0, HEIGHT)
    size = random.uniform(0.5, 2)
    speed = random.uniform(0.2, 0.8)
    stars.append([x, y, size, speed])

# Sound initialization
pygame.mixer.init()

# Background music setup
try:
    pygame.mixer.music.load("bg_music.mp3")
    pygame.mixer.music.set_volume(0.2)  # Lower volume than game sounds (20%)
    pygame.mixer.music.play(-1)  # Loop indefinitely
except:
    pass  # Ignore if music file not found

# Create sound effects using pygame's sound generation
def create_tone(frequency, duration, volume=0.5):
    sample_rate = 22050
    frames = int(duration * sample_rate)
    # Create a proper array with the right shape
    arr = np.zeros((frames, 2), dtype=np.int16)  # Stereo sound
    for i in range(frames):
        wave = int(32767 * volume * math.sin(2 * math.pi * frequency * i / sample_rate))
        arr[i][0] = wave  # Left channel
        arr[i][1] = wave  # Right channel
    return pygame.sndarray.make_sound(arr)

# Create game sounds
try:
    shoot_sound = create_tone(880, 0.1, 0.3)  # Player shoot sound
    enemy_shoot_sound = create_tone(440, 0.1, 0.2)  # Enemy shoot sound
    explosion_sound = create_tone(220, 0.3, 0.4)  # Explosion sound
    powerup_sound = create_tone(1760, 0.2, 0.3)  # Power-up collection sound
    hit_sound = create_tone(330, 0.2, 0.3)  # Player hit sound
    game_over_sound = create_tone(110, 1.0, 0.5)  # Game over sound
except:
    # Fallback if sound creation fails
    class DummySound:
        def play(self): pass
    shoot_sound = DummySound()
    enemy_shoot_sound = DummySound()
    explosion_sound = DummySound()
    powerup_sound = DummySound()
    hit_sound = DummySound()
    game_over_sound = DummySound()

class Player:
    def __init__(self):
        self.width = 50
        self.height = 40
        self.x = WIDTH // 2 - self.width // 2
        self.y = HEIGHT - self.height - 30
        self.speed = 5
        self.health = 100
        self.max_health = 100
        self.shoot_cooldown = 0
        self.shoot_delay = 15
        self.bullets = []
        self.power_level = 1
        self.power_timer = 0
        self.invincible = False
        self.invincible_timer = 0
        
    def draw(self):
        # Draw ship body
        pygame.draw.polygon(screen, PLAYER_COLOR, [
            (self.x + self.width//2, self.y),
            (self.x, self.y + self.height),
            (self.x + self.width, self.y + self.height)
        ])
        
        # Draw cockpit
        pygame.draw.circle(screen, (200, 250, 255), (self.x + self.width//2, self.y + 15), 8)
        
        # Draw engine glow
        if self.invincible and pygame.time.get_ticks() % 200 < 100:
            pygame.draw.polygon(screen, (100, 200, 255, 150), [
                (self.x + self.width//2 - 10, self.y + self.height),
                (self.x + self.width//2 + 10, self.y + self.height),
                (self.x + self.width//2, self.y + self.height + 15)
            ])
        
        # Draw health bar
        pygame.draw.rect(screen, (50, 50, 50), (self.x, self.y - 20, self.width, 10))
        pygame.draw.rect(screen, (0, 255, 0), (self.x, self.y - 20, self.width * (self.health/self.max_health), 10))
        
    def move(self, keys):
        if keys[pygame.K_LEFT] and self.x > 0:
            self.x -= self.speed
        if keys[pygame.K_RIGHT] and self.x < WIDTH - self.width:
            self.x += self.speed
        if keys[pygame.K_UP] and self.y > 0:
            self.y -= self.speed
        if keys[pygame.K_DOWN] and self.y < HEIGHT - self.height:
            self.y += self.speed
            
    def shoot(self):
        if self.shoot_cooldown <= 0:
            shoot_sound.play()  # Play shoot sound
            if self.power_level == 1:
                self.bullets.append(Bullet(self.x + self.width//2 - 2, self.y, 0, -10))
            elif self.power_level == 2:
                self.bullets.append(Bullet(self.x + self.width//2 - 12, self.y, 0, -10))
                self.bullets.append(Bullet(self.x + self.width//2 + 8, self.y, 0, -10))
            elif self.power_level >= 3:
                self.bullets.append(Bullet(self.x + self.width//2 - 2, self.y, 0, -10))
                self.bullets.append(Bullet(self.x + self.width//2 - 15, self.y, -2, -10))
                self.bullets.append(Bullet(self.x + self.width//2 + 11, self.y, 2, -10))
            self.shoot_cooldown = self.shoot_delay
            
    def update(self):
        # Update cooldowns
        if self.shoot_cooldown > 0:
            self.shoot_cooldown -= 1
            
        if self.power_timer > 0:
            self.power_timer -= 1
            if self.power_timer <= 0:
                self.power_level = max(1, self.power_level - 1)
                
        if self.invincible_timer > 0:
            self.invincible_timer -= 1
            if self.invincible_timer <= 0:
                self.invincible = False
                
        # Update bullets
        for bullet in self.bullets[:]:
            bullet.move()
            if bullet.y < 0:
                self.bullets.remove(bullet)
                
    def get_rect(self):
        return pygame.Rect(self.x + 10, self.y + 10, self.width - 20, self.height - 10)

class Bullet:
    def __init__(self, x, y, dx, dy):
        self.x = x
        self.y = y
        self.dx = dx
        self.dy = dy
        self.radius = 4
        self.trail = []
        
    def move(self):
        self.x += self.dx
        self.y += self.dy
        self.trail.append((self.x, self.y))
        if len(self.trail) > 10:
            self.trail.pop(0)
            
    def draw(self):
        # Draw trail
        for i, (trail_x, trail_y) in enumerate(self.trail):
            alpha = int(255 * (i / len(self.trail)))
            radius = int(self.radius * (i / len(self.trail)))
            pygame.draw.circle(screen, (255, 255, 200, alpha), (trail_x, trail_y), radius)
        
        # Draw bullet
        pygame.draw.circle(screen, BULLET_COLOR, (self.x, self.y), self.radius)
        pygame.draw.circle(screen, (255, 255, 255), (self.x, self.y), self.radius - 2)

class Enemy:
    def __init__(self, enemy_type):
        self.type = enemy_type
        self.width = 50
        self.height = 30
        self.x = random.randint(0, WIDTH - self.width)
        self.y = random.randint(-100, -40)
        self.speed = random.uniform(1.0, 3.0) + level * 0.2
        self.health = 1 + level // 3
        self.max_health = self.health
        self.color = ENEMY_COLORS[enemy_type]
        self.shoot_cooldown = random.randint(60, 180)
        self.bullets = []
        self.hit_timer = 0
        self.oscillation = random.uniform(0, 2 * math.pi)  # For flying animation
        self.oscillation_speed = random.uniform(0.05, 0.1)
        
    def move(self):
        self.y += self.speed
        self.oscillation += self.oscillation_speed
        if self.shoot_cooldown > 0:
            self.shoot_cooldown -= 1
        else:
            self.shoot()
            self.shoot_cooldown = random.randint(60, 180)
            
        # Update bullets
        for bullet in self.bullets[:]:
            bullet.move()
            if bullet.y > HEIGHT:
                self.bullets.remove(bullet)
                
    def shoot(self):
        enemy_shoot_sound.play()  # Play enemy shoot sound
        if self.type == 1:  # Shooter enemy
            self.bullets.append(Bullet(self.x + self.width//2, self.y + self.height, 0, 5))
        elif self.type == 2:  # Strafer enemy
            self.bullets.append(Bullet(self.x + self.width//2, self.y + self.height, -2, 5))
            self.bullets.append(Bullet(self.x + self.width//2, self.y + self.height, 2, 5))
            
    def draw(self):
        # Oscillation for flying effect
        osc_offset = math.sin(self.oscillation) * 3
        
        # Draw airplane shape
        # Main body
        body_points = [
            (self.x + self.width//2, self.y + 5 + osc_offset),      # Nose
            (self.x + self.width - 10, self.y + 15 + osc_offset),   # Tail
            (self.x + self.width//2, self.y + 25 + osc_offset),     # Bottom
            (self.x + 10, self.y + 15 + osc_offset)                 # Tail
        ]
        
        # Wings
        wing_points = [
            (self.x + 15, self.y + 10 + osc_offset),
            (self.x + self.width - 15, self.y + 10 + osc_offset),
            (self.x + self.width - 5, self.y + 20 + osc_offset),
            (self.x + 5, self.y + 20 + osc_offset)
        ]
        
        # Tail fin
        tail_points = [
            (self.x + self.width//2, self.y + 25 + osc_offset),
            (self.x + self.width//2 - 8, self.y + self.height + osc_offset),
            (self.x + self.width//2 + 8, self.y + self.height + osc_offset)
        ]
        
        # Cockpit
        cockpit_points = [
            (self.x + self.width//2 - 5, self.y + 10 + osc_offset),
            (self.x + self.width//2 + 5, self.y + 10 + osc_offset),
            (self.x + self.width//2, self.y + 15 + osc_offset)
        ]
        
        # Draw airplane parts
        pygame.draw.polygon(screen, self.color, body_points)
        pygame.draw.polygon(screen, self.color, wing_points)
        pygame.draw.polygon(screen, self.color, tail_points)
        pygame.draw.polygon(screen, (200, 230, 255), cockpit_points)
        
        # Draw details based on type
        if self.type == 0:  # Basic enemy - simple design
            pygame.draw.circle(screen, (255, 255, 255), (self.x + self.width//2, self.y + 12 + osc_offset), 3)
        elif self.type == 1:  # Shooter enemy - gun detail
            pygame.draw.rect(screen, (200, 200, 200), (self.x + self.width//2 - 3, self.y + 20 + osc_offset, 6, 8))
            pygame.draw.circle(screen, (255, 100, 100), (self.x + self.width//2, self.y + 24 + osc_offset), 3)
        elif self.type == 2:  # Strafer enemy - dual guns
            pygame.draw.rect(screen, (200, 200, 200), (self.x + 20, self.y + 20 + osc_offset, 4, 6))
            pygame.draw.rect(screen, (200, 200, 200), (self.x + self.width - 24, self.y + 20 + osc_offset, 4, 6))
            pygame.draw.circle(screen, (100, 255, 100), (self.x + 22, self.y + 23 + osc_offset), 2)
            pygame.draw.circle(screen, (100, 255, 100), (self.x + self.width - 22, self.y + 23 + osc_offset), 2)
            
        # Draw health bar
        if self.hit_timer > 0:
            pygame.draw.rect(screen, (50, 50, 50), (self.x, self.y - 15, self.width, 8))
            pygame.draw.rect(screen, (255, 0, 0), (self.x, self.y - 15, self.width * (self.health/self.max_health), 8))
            self.hit_timer -= 1
            
        # Draw bullets
        for bullet in self.bullets:
            bullet.draw()
            
    def get_rect(self):
        return pygame.Rect(self.x + 5, self.y + 5, self.width - 10, self.height - 5)
        
    def hit(self):
        self.health -= 1
        self.hit_timer = 30
        return self.health <= 0

class PowerUp:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = random.randint(0, 2)  # 0: Health, 1: Weapon, 2: Shield
        self.color = POWERUP_COLORS[self.type]
        self.size = 15
        self.speed = 2
        self.angle = 0
        self.oscillation = 0
        
    def move(self):
        self.y += self.speed
        self.angle += 0.05
        self.oscillation += 0.1
        
    def draw(self):
        # Create airplane shape points
        # Oscillate for flying effect
        osc_offset = math.sin(self.oscillation) * 3
        
        # Main body
        body_points = [
            (self.x, self.y - 5 + osc_offset),           # Nose
            (self.x + 20, self.y + osc_offset),          # Tail
            (self.x, self.y + 5 + osc_offset),           # Bottom
            (self.x - 20, self.y + osc_offset)           # Tail
        ]
        
        # Wings
        wing_points = [
            (self.x - 8, self.y - 3 + osc_offset),
            (self.x + 8, self.y - 3 + osc_offset),
            (self.x + 15, self.y + 3 + osc_offset),
            (self.x - 15, self.y + 3 + osc_offset)
        ]
        
        # Tail fin
        tail_points = [
            (self.x, self.y + 5 + osc_offset),
            (self.x - 5, self.y + 15 + osc_offset),
            (self.x + 5, self.y + 15 + osc_offset)
        ]
        
        # Draw airplane parts
        pygame.draw.polygon(screen, self.color, body_points)
        pygame.draw.polygon(screen, self.color, wing_points)
        pygame.draw.polygon(screen, self.color, tail_points)
        
        # Draw cockpit
        pygame.draw.circle(screen, (200, 255, 255), (self.x, self.y + osc_offset), 3)
        
        # Draw inner symbol based on type
        if self.type == 0:  # Health - red cross
            pygame.draw.rect(screen, (255, 100, 100), (self.x - 6, self.y - 2 + osc_offset, 12, 4))
            pygame.draw.rect(screen, (255, 100, 100), (self.x - 2, self.y - 6 + osc_offset, 4, 12))
        elif self.type == 1:  # Weapon - yellow star
            star_points = []
            for i in range(5):
                angle = i * 2 * math.pi / 5 - math.pi / 2
                radius = 5 if i % 2 == 0 else 2
                x = self.x + radius * math.cos(angle)
                y = self.y + radius * math.sin(angle) + osc_offset
                star_points.append((x, y))
            pygame.draw.polygon(screen, (255, 255, 100), star_points)
        elif self.type == 2:  # Shield - blue shield
            pygame.draw.arc(screen, (100, 200, 255), 
                           (self.x - 6, self.y - 6 + osc_offset, 12, 12), 
                           0, math.pi, 2)
            
    def get_rect(self):
        return pygame.Rect(self.x - 20, self.y - 5, 40, 20)

class Particle:
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.dx = random.uniform(-3, 3)
        self.dy = random.uniform(-3, 3)
        self.size = random.uniform(1, 4)
        self.color = color
        self.life = random.randint(20, 40)
        
    def update(self):
        self.x += self.dx
        self.y += self.dy
        self.life -= 1
        self.size = max(0, self.size - 0.1)
        
    def draw(self):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), int(self.size))

# Create game objects
player = Player()
enemies = []
powerups = []
particles = []

# Font
font = pygame.font.SysFont(None, 36)
big_font = pygame.font.SysFont(None, 72)

# Main game loop
running = True
game_over_sound_played = False  # Track if game over sound has been played
while running:
    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                paused = not paused
                if paused:
                    pygame.mixer.music.pause()
                else:
                    pygame.mixer.music.unpause()
            if event.key == pygame.K_r and game_over:
                # Reset game
                player = Player()
                enemies = []
                powerups = []
                particles = []
                score = 0
                level = 1
                game_over = False
                game_over_sound_played = False  # Reset sound flag
                
    if paused or game_over:
        # Draw UI overlay
        overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
        overlay.fill((0, 0, 0, 150))
        screen.blit(overlay, (0, 0))
        
        if game_over:
            if not game_over_sound_played:
                game_over_sound.play()  # Play game over sound once
                game_over_sound_played = True
                
            game_over_text = big_font.render("GAME OVER", True, (255, 50, 50))
            score_text = font.render(f"Final Score: {score}", True, (255, 255, 255))
            restart_text = font.render("Press R to Restart", True, (200, 200, 100))
            screen.blit(game_over_text, (WIDTH//2 - game_over_text.get_width()//2, HEIGHT//2 - 60))
            screen.blit(score_text, (WIDTH//2 - score_text.get_width()//2, HEIGHT//2 + 10))
            screen.blit(restart_text, (WIDTH//2 - restart_text.get_width()//2, HEIGHT//2 + 60))
        else:
            pause_text = big_font.render("PAUSED", True, (255, 255, 255))
            continue_text = font.render("Press ESC to Continue", True, (200, 200, 100))
            screen.blit(pause_text, (WIDTH//2 - pause_text.get_width()//2, HEIGHT//2 - 30))
            screen.blit(continue_text, (WIDTH//2 - continue_text.get_width()//2, HEIGHT//2 + 30))
            
        pygame.display.flip()
        clock.tick(FPS)
        continue
    
    # Player movement
    keys = pygame.key.get_pressed()
    player.move(keys)
    
    # Shooting
    if keys[pygame.K_SPACE]:
        player.shoot()
        
    # Update player
    player.update()
    
    # Spawn enemies
    if random.randint(1, 60 - min(level, 50)) == 1:
        enemy_type = random.choices([0, 1, 2], weights=[70, 20, 10])[0]
        enemies.append(Enemy(enemy_type))
        
    # Spawn powerups
    if random.randint(1, 500) == 1:
        powerups.append(PowerUp(random.randint(50, WIDTH - 50), -20))
        
    # Update enemies
    for enemy in enemies[:]:
        enemy.move()
        if enemy.y > HEIGHT:
            enemies.remove(enemy)
            
    # Update powerups
    for powerup in powerups[:]:
        powerup.move()
        if powerup.y > HEIGHT:
            powerups.remove(powerup)
            
    # Update particles
    for particle in particles[:]:
        particle.update()
        if particle.life <= 0:
            particles.remove(particle)
            
    # Collision detection - player bullets vs enemies
    for bullet in player.bullets[:]:
        for enemy in enemies[:]:
            if enemy.get_rect().colliderect(bullet.x - bullet.radius, bullet.y - bullet.radius, 
                                           bullet.radius * 2, bullet.radius * 2):
                if bullet in player.bullets:
                    player.bullets.remove(bullet)
                if enemy.hit():
                    explosion_sound.play()  # Play explosion sound
                    # Create explosion particles
                    for _ in range(30):
                        particles.append(Particle(
                            enemy.x + enemy.width//2,
                            enemy.y + enemy.height//2,
                            random.choice(PARTICLE_COLORS)
                        ))
                    enemies.remove(enemy)
                    score += 10 * (enemy.type + 1)
                break
                
    # Collision detection - enemy bullets vs player
    for enemy in enemies:
        for bullet in enemy.bullets[:]:
            if (not player.invincible and 
                player.get_rect().colliderect(bullet.x - bullet.radius, bullet.y - bullet.radius, 
                                             bullet.radius * 2, bullet.radius * 2)):
                if bullet in enemy.bullets:
                    enemy.bullets.remove(bullet)
                hit_sound.play()  # Play hit sound
                player.health -= 10
                player.invincible = True
                player.invincible_timer = 90  # 1.5 seconds of invincibility
                # Create hit particles
                for _ in range(15):
                    particles.append(Particle(
                        player.x + player.width//2,
                        player.y + player.height//2,
                        (255, 100, 100)
                    ))
                if player.health <= 0:
                    game_over = True
                break
                
    # Collision detection - player vs enemies
    for enemy in enemies[:]:
            if (not player.invincible and 
                player.get_rect().colliderect(enemy.get_rect())):
                enemies.remove(enemy)
                hit_sound.play()  # Play hit sound
                player.health -= 20
                player.invincible = True
                player.invincible_timer = 90
                # Create explosion particles
                for _ in range(30):
                    particles.append(Particle(
                        enemy.x + enemy.width//2,
                        enemy.y + enemy.height//2,
                        random.choice(PARTICLE_COLORS)
                    ))
                # Create hit particles
                for _ in range(15):
                    particles.append(Particle(
                        player.x + player.width//2,
                        player.y + player.height//2,
                        (255, 100, 100)
                    ))
                if player.health <= 0:
                    game_over = True
                
    # Collision detection - player vs powerups
    for powerup in powerups[:]:
        if player.get_rect().colliderect(powerup.get_rect()):
            powerups.remove(powerup)
            powerup_sound.play()  # Play power-up sound
            # Create collection particles
            for _ in range(20):
                particles.append(Particle(
                    powerup.x,
                    powerup.y,
                    powerup.color
                ))
                
            if powerup.type == 0:  # Health
                player.health = min(player.max_health, player.health + 30)
            elif powerup.type == 1:  # Weapon
                player.power_level = min(3, player.power_level + 1)
                player.power_timer = 600  # 10 seconds
            elif powerup.type == 2:  # Shield
                player.invincible = True
                player.invincible_timer = 300  # 5 seconds
                
    # Level progression
    if score >= level * 100:
        level += 1
        
    # Drawing
    # Draw background
    screen.fill(BACKGROUND)
    
    # Draw stars
    for star in stars:
        x, y, size, speed = star
        pygame.draw.circle(screen, (200, 200, 255), (int(x), int(y)), size)
        star[1] += speed
        if star[1] > HEIGHT:
            star[1] = 0
            star[0] = random.randint(0, WIDTH)
    
    # Draw particles
    for particle in particles:
        particle.draw()
        
    # Draw powerups
    for powerup in powerups:
        powerup.draw()
        
    # Draw enemies
    for enemy in enemies:
        enemy.draw()
        
    # Draw player bullets
    for bullet in player.bullets:
        bullet.draw()
        
    # Draw player
    player.draw()
    
    # Draw UI
    pygame.draw.rect(screen, UI_BG, (10, 10, 200, 40), border_radius=5)
    score_text = font.render(f"Score: {score}", True, UI_COLOR)
    screen.blit(score_text, (20, 15))
    
    pygame.draw.rect(screen, UI_BG, (WIDTH - 160, 10, 150, 40), border_radius=5)
    level_text = font.render(f"Level: {level}", True, UI_COLOR)
    screen.blit(level_text, (WIDTH - 150, 15))
    
    # Draw power level indicator
    if player.power_level > 1:
        power_text = font.render(f"Weapon Lvl: {player.power_level}", True, (255, 215, 0))
        screen.blit(power_text, (WIDTH//2 - power_text.get_width()//2, 15))
        
    if player.invincible:
        shield_text = font.render("SHIELD ACTIVE", True, (100, 255, 255))
        screen.blit(shield_text, (WIDTH//2 - shield_text.get_width()//2, HEIGHT - 40))
    
    # Draw instructions
    if score < 50:
        help_text = font.render("Arrow Keys: Move, Space: Shoot, ESC: Pause", True, (150, 150, 200))
        screen.blit(help_text, (WIDTH//2 - help_text.get_width()//2, HEIGHT - 30))
    
    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
sys.exit()


SystemExit: 